## makemore: becoming a backprop ninja

swole doge style

In [1]:
# there no change change in the first several cells from last lecture

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
# ok biolerplate done, now we get to the action:

In [7]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [8]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [9]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [10]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3291, grad_fn=<NegBackward0>)

In [33]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# loss = -(a+b+c...)*(1/N)  where a,b,c... are scalars in different rows of logprobs
# dloss/da = 1/N, so gradient of logprobs is given as follows:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n

# logprobs = log(probs) -> dlogprobs/da_ij = 1/a_ij for all a_ij in probs)
dprobs = probs**-1 * dlogprobs # don't forget the chain-rule

# probs_ij = a_ij / (a_i1 + a_i2 +... + a_iM) for each row i
# Since counts_sum_inv_i = 1/(a_i1 + a_i2 +... + a_iM) 
# and probs=counts*counts_sum_inv broadcasts counts_sum_inv, 
# we just sum up the gradient of the 'copies' of counts_sum_inv
# dprobs/dcounts_sum_inv_i = sum(a_ij) for all j
dcounts_sum_inv = (counts * dprobs).sum(dim=1, keepdim=True)

# counts_sum_inv = 1/counts_sum -> dcounts_sum_inv/dcounts_sum = -1/counts_sum^2
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv

# counts_sum_i = a_i1 + ... a_iN -> dcounts_sum/counts_ij = 1 for all values of i,j
dcounts = torch.ones_like(counts) * dcounts_sum
# similarly, the gradient of each term in counts[32x27] is just the corresponding scalar from counts_sum_inv
dcounts += torch.ones_like(counts) * counts_sum_inv *  dprobs

# counts = e^norm_logits -> dcounts/dnorm_logits = e^dnorm_logits
dnorm_logits = counts * dcounts

# norm_logits = logits[32,27] - logits_maxes[32] 
# dnorm_logits/dlogit_maxes_i = sum_row_i(-1 dnorm_logits_ij) 
# again, we sum up every 'copy' that results from broadcasting
dlogit_maxes = (-torch.ones_like(logit_maxes) * dnorm_logits).sum(dim=1, keepdims=True)

# dnorm_logits/dlogits = 1 * dnorm_logits
dlogits = dnorm_logits.clone()

# dlogit_maxes/dlogits = 1 * dlogit_maxes - for the entries that correspond to logit_maxes, 0 otherwise
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

# logits[batch_size, 27] = h[batch_size, n_hidden] @ W2[n_hidden, 27] + b[1, batch_size]
# for each h_#j, there is a corresponding W2_j -> dlogits/dh_ij = sum(W_jk) for all k 
# we can get the same result with the following matrix multiplication:
dh = dlogits @ W2.T

# Likewise, for each W_ji, there is a corresponding h_ij, so dlogits/dW_ji = sum(h_ki) for all k
dW2 = h.T @ dlogits

# dlogits/db_i = sum ith column of the partial derivative matrix
# db2 = (torch.zero_like(b2) * dlogits).sum(0) # don't forget chain rule
db2 = dlogits.sum(0, keepdims=True) 

# h = tanh(hpreact) -> dh/dhpreact = sech^2(hpreact)
# Property: sech^2(x) = 1 - tanh^2(x)
dhpreact = (1.0 - h**2) * dh

# hpreact = bngain * bnraw + bnbias
# dhpreact/dbngain_[i] = sum(bnraw_ji) for all j
dbngain = (bnraw * dhpreact).sum(0, keepdims=True)

# dhpreact/dbnraw_ij = bngain[j] for all i
dbnraw = bngain * dhpreact

#dhpreact/dbnbias_i = sum ith column of the partial derivatives matrix
# dbnbias = (torch.zero_like(bnbias) * dhpreact).sum(0)
dbnbias = dhpreact.sum(0, keepdims=True)

#bnraw = bndiff * bnvar_inv -> bndiff * bnvar_inv
# dbnraw/bndiff_i = bnvar_inv broadcasted along 0 dimension
dbndiff = bnvar_inv * dbnraw
# dbnraw/dbnvar_inv_i = sum ith column of the partial derivatives matrix
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdims=True)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# dbnvar_inv/dbnvar = just the derivative of sqrt(x)
dbnvar = -0.5*((bnvar + 1e-5)**-1.5) * dbnvar_inv
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
dbndiff += (2*bndiff) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnbias          | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
bnraw  

### Exercise 1 Notes
When a vector is broadcasted to operate on a matrix, then the gradient of each entry in the vector is the sum of the gradients of each broadcasted operation.
For example, if we have a $n \times m$ matrix $M$ and a $1\times m$ row vector $v$, then $M \cdot v$ is equivalent to broadcasting $v$ about the $0$-th dimension $n$ times until we have a $n \times m$ matrix $V$ which we then pairwise multiply with $M$ to get $Z=M \cdot V$. Notice that $v_i$ is repeated as $V_{1i}, V_{2i},\dots,V_{ni}$, so the partial derivative of $Z$ with respect to $v_i$ is actually the sum of the partial derivatives of $Z$ with respect to $V_{1i}, V_{2i},\dots,V_{ni}$.

Because of broadcasting rules, most of the `torch.ones_like()` calls made above are actually unnecessary.